# Predict female behav vars from surrogate neural population activity

In [2]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import linear_model
import sys

from disp import set_plot
from my_torch import skl_fit_ridge

cc = np.concatenate


TARG_BHV = 'MTN'
TWDWS = [.03, 1, 60]
TARGS = [f'{TARG_BHV}_MN_{twdw}' for twdw in TWDWS]

PFX_BHV = f'data/simple/behav_xtd/behav_xtd'

NNRN = 224
NRL_MDL = 'ma'

PTRAIN = .8
NSPLIT = 30

ALPHA = 10

NSPLIT_SAVE_Y_HAT = 5

MASK_PFX = 'data/simple/masks/mask'

if MASK_PFX:
    FSAVE_SFX = f'{TARG_BHV.lower()}_ridge_alpha_{ALPHA}_masked'
else:
    FSAVE_SFX = f'{TARG_BHV.lower()}_ridge_alpha_{ALPHA}'

In [2]:
# SPECIFY STRAINS USED
STRAINS = ['NM91', 'ZH23']
STRAIN_KEY = '_'.join(STRAINS).lower()

FSTRAIN = 'data/simple/strains.csv'
MSTRAINS = [(pd.read_csv(FSTRAIN)['STRAIN'] == strain) for strain in STRAINS]
MSTRAIN = np.any(MSTRAINS, axis=0)
ISTRAIN = MSTRAIN.nonzero()[0]

NTR = len(ISTRAIN)
NTRAIN = int(round(PTRAIN*NTR))

In [4]:
pfx_nrl = f'data/simple/mlv/neur_basic/baker_{NRL_MDL}/mlv_baker_{NRL_MDL}'

r_cols_use = [f'R_{inrn}' for inrn in range(NNRN)]

fsave = f'data/simple/mlv/neur_basic/baker_{NRL_MDL}_{STRAIN_KEY}_{FSAVE_SFX}.npy'

# fit regression models
rslts = skl_fit_ridge(
    pfxs=[pfx_nrl, PFX_BHV],
    cols_x=r_cols_use,
    targs=TARGS,
    itr_all=ISTRAIN, 
    ntrain=NTRAIN,
    nsplit=NSPLIT,
    mask_pfx=MASK_PFX,
    return_y=np.arange(NSPLIT_SAVE_Y_HAT),
    alpha=ALPHA)

Loading...

Split 0
Split 1
Split 2
Split 3
Split 4
Split 5
Split 6
Split 7
Split 8
Split 9
Split 10
Split 11
Split 12
Split 13
Split 14
Split 15
Split 16
Split 17
Split 18
Split 19
Split 20
Split 21
Split 22
Split 23
Split 24
Split 25
Split 26
Split 27
Split 28
Split 29

In [6]:
itr_trains = [rslt.itr_train for rslt in rslts]
itr_tests = [rslt.itr_test for rslt in rslts]

np.save('data/simple/mlv/train_test_splits.npy', np.array([{'itr_trains': itr_trains, 'itr_tests': itr_tests}]))

In [9]:
data = np.load('data/simple/mlv/train_test_splits.npy', allow_pickle=True)[0]

for ctr, (itr_train, itr_test) in enumerate(zip(data['itr_trains'], data['itr_tests'])):
    print('Split', ctr)
    print('Train', itr_train)
    print('Test', itr_test)

Split 0
Train [  2  13 205  41 218  30  45  43 228 238   7  26  33 215   8  16  24 208
 227  42  22   6 213 200 221 206 223 229 232   3 214 225  27  18 202 203
 224 211 207   4  15  17  40  38   5 231 220   0  34  28  11  35  23 204
  10  31 212 209 230   1  32 226  14 234  19  29 201 236 237  20]
Test [233  25  37  46  39 217 210  12 222  36  21   9 235 219 216 199  44]
Split 1
Train [199   7 209 223  17 222 208  15  25   6  33  29   9   5  18 201 216  28
  37 230 205 212 215   4 233 220  12  39 217 213  44  26 219  31 218   8
 207 234  45 232 204  24  30  13 206 231  19  38 227  32  16 203 238  11
  40 228  22 226 235   1 224  41  14  27 236  20  46  35 214   2]
Test [211  23 210  43  10 237 225  21 229  42   3 200  34  36 202   0 221]
Split 2
Train [ 17  34   6  15 207  36  33 234 209 235 229 211  30 203  16  14  22 202
 215   0   4   8   5 233 238 225 205  12 224  21 200 216   7 204 212 230
 236 210  26  45 201  42   9  39 221 220  29 218  46  23  25  19  27   2
 214 199  28  37 21